# Load PointClouds in Spark using PDAL
We calculate the Normalized Height for each point.

In [55]:
import geotrellis.pointcloud.spark.Extent3D
import geotrellis.pointcloud.spark.io.hadoop.HadoopPointCloudRDD
import io.pdal.Pipeline
import io.pdal.pipeline.{FerryFilter, HagFilter, LasRead, LasWrite}
import geotrellis.spark.io.hadoop.HadoopCollectionLayerReader
import org.apache.hadoop.fs.Path
import org.apache.spark.{SparkConf, SparkContext}
import org.apache.spark.rdd.RDD
import spire.syntax.cfor._

In [2]:
implicit val sparkContext = sc

Waiting for a Spark session to start...

sparkContext = org.apache.spark.SparkContext@202e8b84


Spark app-20180418094748-0000: Some(http://145.100.58.104:4040)

## Data data paths

In [105]:
val tiles_path = "/data/local/home/ecolidar/"
val tile = "C_25EZ2"
val laz_filepath = tiles_path + tile + ".laz"

tiles_path = /data/local/home/ecolidar/
tile = C_25EZ2
laz_filepath = /data/local/home/ecolidar/C_25EZ2.laz


/data/local/home/ecolidar/C_25EZ2.laz

### Read local and save the result back to a file

In [104]:
val las_pipelineExpr = LasRead(las_filepath) ~ HagFilter() ~ FerryFilter(dimensions = "Z=HeightAboveGround") ~ LasWrite(tiles_path + tile + "_hag.laz")
val las_pipeline: Pipeline = las_pipelineExpr.toPipeline
las_pipeline.execute()

las_pipelineExpr = List(LasRead(/data/local/home/ecolidar/C_25EZ2.las,None,None,None,None,None,readers.las), HagFilter(filters.hag), FerryFilter(Z=HeightAboveGround,filters.ferry), LasWrite(/data/local/home/ecolidar/C_25EZ2_hag.laz,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,writers.las))
las_pipeline = io.pdal.Pipeline@4ac1c789


io.pdal.Pipeline@4ac1c789

### Verify if the saved file has HeightAboveGround

### Read HDFS and apply HagFilter

In [86]:
val laz_path = new Path("/user/hadoop/ahn3/C_25EZ2.laz")
val pipelineExpr = Read("local") ~ HagFilter() ~ FerryFilter(dimensions = "HeightAboveGround=Z")
val rdd_laz = HadoopPointCloudRDD(laz_path, options = HadoopPointCloudRDD.Options(pipeline = pipelineExpr))

las_path = /user/hadoop/ahn3/C_25EZ2.las
laz_path = /user/hadoop/ahn3/C_25EZ2.laz


/user/hadoop/ahn3/C_25EZ2.laz

### Verify if HeightAboveGround shows up in the schema

In [97]:
rdd_laz.map{ case (h, i) => h.schema}.take(1)

Array("{
  "schema":
  {
    "dimensions":
    [
      {
        "name": "X",
        "size": 8,
        "type": "floating"
      },
      {
        "name": "Y",
        "size": 8,
        "type": "floating"
      },
      {
        "name": "Z",
        "size": 8,
        "type": "floating"
      },
      {
        "name": "Intensity",
        "size": 2,
        "type": "unsigned"
      },
      {
        "name": "ReturnNumber",
        "size": 1,
        "type": "unsigned"
      },
      {
        "name": "NumberOfReturns",
        "size": 1,
        "type": "unsigned"
      },
      {
        "name": "ScanDirectionFlag",
        "size": 1,
        "type": "unsigned"
      },
      {
        "name": "EdgeOfFlightLine",
        "size": 1,
        "type": "unsigne...


[{
  "schema":
  {
    "dimensions":
    [
      {
        "name": "X",
        "size": 8,
        "type": "floating"
      },
      {
        "name": "Y",
        "size": 8,
        "type": "floating"
      },
      {
        "name": "Z",
        "size": 8,
        "type": "floating"
      },
      {
        "name": "Intensity",
        "size": 2,
        "type": "unsigned"
      },
      {
        "name": "ReturnNumber",
        "size": 1,
        "type": "unsigned"
      },
      {
        "name": "NumberOfReturns",
        "size": 1,
        "type": "unsigned"
      },
      {
        "name": "ScanDirectionFlag",
        "size": 1,
        "type": "unsigned"
      },
      {
        "name": "EdgeOfFlightLine",
        "size": 1,
        "type": "unsigned"
      },
      {
        "name": "Classification",
        "size": 1,
        "type": "unsigned"
      },
      {
        "name": "ScanAngleRank",
        "size": 4,
        "type": "floating"
      },
      {
        "name": "Use

### Load all the points into RDD and compare Z with NormalizedHeight

In [ ]:
val points :RDD[(Double, Double, Double, Byte, Float)] = rdd_laz.flatMap(_._2).mapPartitions{ _.map { packedPoints =>

  var res = new Array[(Double, Double, Double, Byte, Float)](packedPoints.length)
  cfor(0)(_ < packedPoints.length, _ + 1) { i =>
    res(i) = (packedPoints.getX(i), packedPoints.getY(i), packedPoints.getZ(i), packedPoints.getByte(i, dim = "Classification"), packedPoints.getFloat(i, dim = "HeightAboveGround"))
  }
  res
}}.flatMap( m => m).filter(_._4 != 2).filter(_._3 > 0)
points2.take(10)

### Verify if the created file and uploaded to HDFS has Normalized Height

In [106]:
val laz_path = new Path("/user/hadoop/ahn3/C_25EZ2_hag.laz")
val rdd_laz = HadoopPointCloudRDD(laz_path)
rdd_laz.cache()
rdd_laz.count()

laz_path = /user/hadoop/ahn3/C_25EZ2_hag.laz
rdd_laz = NewHadoopRDD[152] at newAPIHadoopRDD at HadoopPointCloudRDD.scala:76


1

In [107]:
val points :RDD[(Double, Double, Double, Byte)] = rdd_laz.flatMap(_._2).mapPartitions{ _.map { packedPoints =>
  var res = new Array[(Double, Double, Double, Byte)](packedPoints.length)
  cfor(0)(_ < packedPoints.length, _ + 1) { i =>
    res(i) = (packedPoints.getX(i), packedPoints.getY(i), packedPoints.getZ(i), packedPoints.getByte(i, dim = "Classification"))
  }
  res
}}.flatMap( m => m).filter(_._4 != 2).filter(_._3 > 0)
val points10 = points.take(10)

points = MapPartitionsRDD[157] at filter at <console>:100


MapPartitionsRDD[157] at filter at <console>:100